In [79]:
import requests as rq
from bs4 import BeautifulSoup
from IPython.display import HTML
import pandas as pd
from tqdm import tqdm_notebook
import re

In [55]:
links = []
for o in tqdm_notebook("ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"):
    url_list= "http://bvmf.bmfbovespa.com.br/cias-Listadas/Empresas-Listadas/BuscaEmpresaListada.aspx?Letra={}&idioma=pt-br".format(o)
    soup = BeautifulSoup(rq.get(url_list).content, 'lxml')
    for a in soup.find_all('a'):
        if a.attrs['href'].startswith("ResumoEmpresaPrincipal.aspx"):
            links.append((a.text,a.attrs['href']))

In [ ]:
html_data = []

In [79]:
downloaded = set((i['company'] for i in html_data))
for company, link in tqdm_notebook(links):
    if company in downloaded:
        continue
    codigocvm = re.search(r"codigoCvm=(\d+)", link).expand(r'\1')
    url = "http://bvmf.bmfbovespa.com.br/cias-Listadas/Empresas-Listadas/ResumoEventosCorporativos.aspx?codigoCvm={cvm}&tab=3&idioma=pt-br".format(cvm=codigocvm)
    html = rq.get(url).content.decode('utf-8')
    html_data.append({
        'company':company,
        'link':link,
        'codigoCvm':codigocvm,
        'content':html})

In [3]:
import pickle 

with open("../Dados/html_split_proventos.pkl", 'wb') as w:
    pickle.dump(html_data, w)

In [4]:
with open("../Dados/html_split_proventos.pkl", 'rb') as r:
    html_data = pickle.load(r)

In [89]:
grupamentos = []
for i in tqdm_notebook(html_data):
    try:
        company_data = dict(pd.read_html(i['content'])[0].to_records(index=False))
        grupamento = pd.read_html(i['content'], match='Fator de Grupamento',  decimal='.')[0]
        grupamento['TICKER'] = company_data['Código:']
        grupamentos.append(grupamento)
    except ValueError:
        pass
    except:
        raise
grupamentos = pd.concat(grupamentos).drop_duplicates()
grupamentos['Deliberado em'] = pd.to_datetime(grupamentos['Deliberado em'], dayfirst=True, errors='coerce')
grupamentos['Negócios com até'] = pd.to_datetime(grupamentos['Negócios com até'], dayfirst=True, errors='coerce')

In [101]:
grupamentos.to_parquet("../Dados/grupamentos.parquet")

,Proventos,Código ISIN,Deliberado em,Negócios com até,% / Fator de Grupamento,Ativo Emitido,Observações,TICKER
2,GRUPAMENTO,BREALTACNOR4,NaT,20/12/1991,0.001,BREALTACNOR4,NaN,EALT
3,GRUPAMENTO,BREALTACNPR1,NaT,20/12/1991,0.001,BREALTACNPR1,NaN,EALT
4,GRUPAMENTO,BREALTACPOR9,NaT,20/12/1991,0.001,BREALTACNOR4,NaN,EALT
5,GRUPAMENTO,BREALTACPPR6,NaT,20/12/1991,0.001,BREALTACNPR1,NaN,EALT
